In [320]:
import pandas as pd
import numpy as np
import sqlite3
import re

conn = sqlite3.connect("profiles.db")
cursor = conn.cursor()
cursor.execute("ATTACH DATABASE 'colleges.db' AS colleges")

In [321]:
df = pd.read_sql_query("SELECT * FROM colleges.university_profiles AS c JOIN profiles AS p ON c.school = p.school",conn)
df_w_admission_rate = df.copy(deep=True)

In [322]:
def get_type_map(df_column):
    return {t:i for i, t in enumerate(set(df_column))}

def convert_sat_to_act(sat):
    if np.isnan(sat):
        return np.nan
    sat_scores = np.array([990, 1060, 1140, 1210, 1270, 1330, 1390, 1450, 1510, 1560, 1620, 1680, 1740, 1800, 1860, 1920, 1980, 2020, 2080, 2140, 2220, 2290, 2380, 2410])
    return np.where(sat < sat_scores)[0][0] + 13
    
def combine_test_scores(x):
    if not np.isnan(x['act']):
        return x['act']/36.0
    sat = np.array([x['sat_m'], x['sat_r'], x['sat_w']])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def convert_string(x):
    try:
        return int(x)
    except:
        return np.nan

def combine_avg_test_scores(x):
    if not np.isnan(convert_string(x['avg_act'])):
        return convert_string(x['avg_act'])/36.0
    sat = np.array([convert_string(x['sat_math']), convert_string(x['sat_reading']), convert_string(x['sat_writing'])])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def map_statuses(x):
    status_map = {"Denied": 0, "Deferred": 0, "Wait-Listed": 0, "Accepted": 2, "Will Attend": 2}
    return status_map[x['status']]

def convert_class_rank(x):
    s = str(x['class_rank'])
    if " of " in s:
        return float(s[:s.index(" of ")])/float(s[s.index(" of ") + 4:])
    if "Top " in s:
        return float(s[4:s.index("%")])/100.0
    if "Bottom " in s:
        return float(s[7:s.index("%")])/100.0
    return np.nan

def convert_instate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        instate = x['cost_attendance'].split("<br>")[0]
        return instate[instate.index("$")+1:]
    else:
        return x['cost_attendance']

def convert_outstate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        outstate = x['cost_attendance'].split("<br>")[1]
        return outstate[outstate.index("$")+1:]
    else:
        return x['cost_attendance']

def get_tuition(x):
    return x['in_state_tuition'] if x['state'] == x['hs_state'] else x['out_state_tuition']

def map_hs_types(x):
    hs_type_map = {"Public": 0, "Private": 1, "Parochial": 2, "Home": 3}
    return hs_type_map[x['hs_type']]

def map_ins_types(x):
    ins_type_map = {"Public": 0, "Private": 1, "Private for-profit": 1, "&nbsp;": np.nan}
    return ins_type_map[x['institution_type']]

In [323]:
df = df.loc[:,~df.columns.duplicated()]
df.fillna(value=pd.np.nan, inplace=True)
df['avg_act'] = df['avg_act'].map(lambda x: str(x).strip('-'))
df['test_score'] = df.apply(combine_test_scores, axis=1)
df['avg_test_score'] = df.apply(combine_avg_test_scores, axis=1)
df['class_rank'] = df.apply(convert_class_rank, axis=1)
df['institution_type'] = df.apply(map_ins_types, axis=1)
df['in_state_tuition'] = df.apply(convert_instate_tuition, axis=1)
df['out_state_tuition'] = df.apply(convert_outstate_tuition, axis=1)
#df['tuition'] = df.apply(get_tuition, axis=1)

for col in ['cost_attendance', 'city', 'sat_m', 'sat_r', 'sat_w', 'act', 'sat_math', 'sat_reading', 'sat_writing', 'avg_act', 'gpa_w']:
    df = df.drop(col, axis=1)

df = df.dropna()#subset=['test_score','avg_test_score','gpa_uw','class_rank','hs_state','hs_type','institution_type'])

df['status'] = df.apply(map_statuses, axis=1)
df['hs_type'] = df.apply(map_hs_types, axis=1)
replace_columns = ['hs_state', 'state', 'gender']
for col in replace_columns:
    df[col].replace(get_type_map(df[col]), inplace=True)

In [324]:
print(len(df.index))

44042


In [325]:
whole_df = df

def split_df_on(feature=None):
    split_dfs = {}
    if feature:
        for val in whole_df[feature].unique():
            split_dfs[val] = (whole_df.loc[whole_df[feature]==val])
    else:
        split_dfs = {'all': whole_df}
    return split_dfs
df.sample(n=100)

,school,state,avg_gpa,average_freshman_aid,admission_rate,faculty_total,international_percent,institution_type,female_percentage,year,...,gpa_uw,class_rank,status,eaed,legacy,athlete,test_score,avg_test_score,in_state_tuition,out_state_tuition
20625,Western New England University,17,3.40,26141,81,241,3.3,1.0,37.1,2017,...,3.50,0.250000,2,0,0,0,0.694444,0.666667,52547,52547
36440,University of Alabama,18,3.72,16254,53,1382,1.6,0.0,55.9,2019,...,3.55,0.222756,2,0,0,0,0.944444,0.750000,26572,45022
34084,Harvard College,17,4.18,55354,5,975,11.,1.0,47.7,2019,...,3.90,0.104348,0,0,0,0,0.805556,0.944444,69600,69600
73047,Elon University,8,4.00,19885,67,433,2.3,1.0,59.6,2017,...,3.83,0.039326,2,1,0,0,0.861111,0.750000,51048,51048
42778,"University of California, Irvine",21,3.97,23886,37,1290,16.,0.0,52.7,2009,...,3.83,0.019178,2,0,0,0,0.888889,0.805556,33826,61840
90981,Wheaton College,30,3.72,27369,85,222,2.9,1.0,54.1,2012,...,3.20,0.250000,0,0,0,0,0.916667,0.805556,49300,49300
23896,Mount St. Mary College,4,3.30,23758,93,89,0.5,1.0,70.2,2018,...,3.30,0.141026,2,0,0,0,0.583333,0.583333,49076,49076
71070,Fordham University,4,3.65,36410,46,735,8.5,1.0,57.5,2017,...,3.40,0.175439,0,0,0,0,0.611111,0.805556,72239,72239
82871,Virginia Polytechnic Institute and State Unive...,35,3.97,17549,70,1806,6.4,0.0,43.1,2015,...,3.90,0.005168,2,0,0,0,0.972222,0.750000,21920,39099
44816,Loyola Marymount University,21,3.75,28854,52,585,9.8,1.0,55.6,2013,...,3.50,0.146862,2,0,0,0,0.805556,0.777778,68764,68764


In [327]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression

split_on = None
for val, df_s in split_df_on(split_on).items():
    schools = df_s['school']
    k = 1
    
    df0 = df_s[df.status == 0]
    df2 = df_s[df.status == 2].sample(n = int(k*len(df0)))
    
    df_final = pd.concat([df0, df2])
    df_final = df_final.drop('school', axis=1)
    X = df_final.drop('status', axis=1)
    y = df_final['status']
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    mlp = MLPClassifier(hidden_layer_sizes=(21, 21, 21),max_iter=1000)
    mlp.fit(X_train,y_train)
    predictions = mlp.predict(X_test)
    print(confusion_matrix(y_test,predictions))
    print(classification_report(y_test,predictions))

    logisticRegr = LogisticRegression()
    logisticRegr.fit(X_train, y_train)
    score = logisticRegr.score(X_test, y_test)
    print(f"Logistic Regression Score for {split_on} {val}: " + str(score))



c:\users\peaka\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\peaka\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:23: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
c:\users\peaka\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.


[[1839  410]
 [ 474 1672]]
              precision    recall  f1-score   support

           0       0.80      0.82      0.81      2249
           2       0.80      0.78      0.79      2146

   micro avg       0.80      0.80      0.80      4395
   macro avg       0.80      0.80      0.80      4395
weighted avg       0.80      0.80      0.80      4395

Logistic Regression Score for None all: 0.7995449374288964


c:\users\peaka\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [328]:
old_x = X.copy()
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

from collections import defaultdict
experimental_acceptance_rates = defaultdict(lambda: defaultdict(int))
print(set(schools))
for row, school in zip(X, schools):
    res = mlp.predict([row])[0]
    if res == 2:
        experimental_acceptance_rates[school]['reject'] += 1
    else:
        experimental_acceptance_rates[school]['accept'] += 1



    

c:\users\peaka\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\peaka\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


{'Bethany Lutheran College', 'Tuskegee University', 'University of Illinois at Chicago', 'Northwestern State University of Louisiana', 'Fairleigh Dickinson University - Metropolitan Campus', 'Saint Anselm College', 'Western Kentucky University', 'Samford University', 'Adams State University', 'Bridgewater State University', 'Bloomsburg University of Pennsylvania', 'Nyack College', 'West Virginia Wesleyan College', 'Webster University', 'American University', 'Eastern New Mexico University', 'Towson University', 'St. Olaf College', 'State University of New York College at Geneseo', 'Northern Illinois University', 'Saint Leo University', 'Hunter College', 'Ohio Valley University', 'Manhattan College', 'Simpson University', 'University of Massachusetts Dartmouth', 'Bemidji State University', 'Marshall University', 'Bridgewater College', "St. Mary's University", 'Hamline University', 'Marietta College', 'College of Mount St. Joseph', 'Coker College', 'Salisbury University', 'Wayne State Un

In [329]:
experimental_acceptance_rates

defaultdict(<function __main__.<lambda>()>,
            {'Bryn Athyn College': defaultdict(int, {'accept': 1}),
             'Adelphi University': defaultdict(int,
                         {'accept': 25, 'reject': 24}),
             'Albany College of Pharmacy and Health Sciences': defaultdict(int,
                         {'accept': 9, 'reject': 2}),
             'Albertus Magnus College': defaultdict(int,
                         {'reject': 1, 'accept': 1}),
             'Albright College': defaultdict(int, {'accept': 32, 'reject': 5}),
             'Alfred University': defaultdict(int,
                         {'accept': 11, 'reject': 9}),
             'Allegheny College': defaultdict(int,
                         {'reject': 27, 'accept': 30}),
             'DeSales University': defaultdict(int,
                         {'reject': 8, 'accept': 9}),
             'Alvernia University': defaultdict(int, {'accept': 8}),
             'American International College': defaultdict(int, {'a

In [330]:
compare_vs_ground_truth = defaultdict(str)

for (index, row), school in zip(old_x.iterrows(), schools):
    accept_rate = None
    if experimental_acceptance_rates[school]['accept'] != 0 and experimental_acceptance_rates[school]['accept'] != 0:
        accept_rate = experimental_acceptance_rates[school]['accept']/(experimental_acceptance_rates[school]['accept']+experimental_acceptance_rates[school]['reject'])
    elif experimental_acceptance_rates[school]['accept'] == 0:
        accept_rate = 0
    elif experimental_acceptance_rates[school]['reject'] == 0:
        accept_rate = 1
    
    compare_vs_ground_truth[school] = [accept_rate, int(row['admission_rate'])/100]

In [331]:
compare_vs_ground_truth

defaultdict(str,
            {'Bryn Athyn College': [1.0, 0.67],
             'Adelphi University': [0.5102040816326531, 0.79],
             'Albany College of Pharmacy and Health Sciences': [0.8181818181818182,
              0.43],
             'Albertus Magnus College': [0.5, 0.43],
             'Albright College': [0.8648648648648649, 0.43],
             'Alfred University': [0.55, 0.5],
             'Allegheny College': [0.5263157894736842, 0.5],
             'DeSales University': [0.5294117647058824, 0.64],
             'Alvernia University': [1.0, 0.09],
             'American International College': [1.0, 0.09],
             'Saint Anselm College': [1.0, 0.09],
             'St. Bonaventure University': [1.0, 0.09],
             'Assumption College': [1.0, 0.09],
             'Saint Francis University': [0.8571428571428571, 0.09],
             'College of the Atlantic': [1.0, 0.09],
             'St. John Fisher College': [1.0, 0.09],
             "St. John's University": [1.0, 